***Syntax natürlicher Sprachen, WS 2022/23***

---
# Übung 8 (Lösung Präsenzaufgaben)


In [1]:
from nltk.tree import Tree
import nltk
import sys

In [2]:
import spacy
from spacy import displacy

In [3]:
def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])

In [4]:
from nltk import DependencyGraph
from nltk.tree import Tree
from itertools import chain


### tree_labeled-Methode für DependencyGraphs (pretty_print-Dependenzbäume mit Labels):
def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

    
DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

In [5]:
#Konvertierungsfunktion von conll(4)-Format (MALT-tabular) für Rendering mit displaCy:
def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex

#html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

In [6]:
def show_dep_trees(sent_nr):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)

    tree_labeled = dg.tree_labeled()
    tree_labeled.pretty_print(unicodelines=True)   
    
    ex = displacy_dep_input(sent)
    html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

--- 
## Aufgabe 1: Syntaxregeln komplexer Sätze
#### Geben Sie zu den folgenden CFG-Outputs des Stanford-Parsers (trainiert auf Penn-Treebank) anhand der Ableitungsbäume die Regeln der jeweils auftretenden komplexen Satzkonstruktionen an und benennen Sie den Satz- bzw. Konstruktionstyp. 

#### Analysieren Sie anschließend die Dependenzstruktur der Sätze im UD-Schema und geben Sie dann explizit die verwendeten Dependenzregeln für die komplexen Satzkonstruktionen an.


- Penn-Treebank Tagset: http://surdeanu.cs.arizona.edu//mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html
- UD Dependency Tagset: https://universaldependencies.org/u/dep/all.html

---
### Aufgabe 1a: 

In [7]:
sentence = "I believe that you win."

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

Objektsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

Komplementsatz

### *Konstituentenanalyse:*

In [8]:
tree_str = '(ROOT (S (NP (PRP I)) (VP (VBP believe) (SBAR (IN that) (S (NP (PRP you)) (VP (VBP win))))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

            ROOT                     
             │                        
             S                       
 ┌───────────┴────┬────────────────┐  
 │                VP               │ 
 │     ┌──────────┴───┐            │  
 │     │             SBAR          │ 
 │     │     ┌────────┴────┐       │  
 │     │     │             S       │ 
 │     │     │        ┌────┴───┐   │  
 NP    │     │        NP       VP  │ 
 │     │     │        │        │   │  
PRP   VBP    IN      PRP      VBP  . 
 │     │     │        │        │   │  
 I  believe that     you      win  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`VP -> VBP SBAR`

`SBAR -> IN S`

---
### *Dependenzanalyse:*

In [9]:
sent_nr = """
1 I 2 nsubj  
2 believe 0 ROOT
3 that 5 mark
4 you 5 nsubj
5 win 2 ccomp
"""

show_dep_trees(sent_nr)

         believe(ROOT)                      
   ┌───────────┴───────────┐                 
   │                   win(ccomp)           
   │           ┌───────────┴──────────┐      
I(nsubj)   that(mark)             you(nsubj)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`believe -> win (ccomp)`

`win -> that (mark)`

---
### Aufgabe 1b: 

In [10]:
sentence = "I want to believe."

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

Infinitiv-Objektsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

Komplementsatz

### Zusatzfragen:

#### Welche Form der Kontrolle liegt hier vor? (Subjekt- oder Objektkontrolle?)

Subjektkontrolle (Subjekt des Matrixsatzes ist Subjekt des Infinitivsatzes)

#### Welche Form der Kontrolle liegt in folgendem Satz vor: *Sie überreden ihn, nach München zu fahren?*

Objektkontrolle (Objekt des Matrixsatzes ist Subjekt des Infinitivsatzes)


#### Welche satzwertigen Funktionen können Infinitivkonstruktionen erfüllen? 

- **Objektsatz (`xcomp`):**  *sie plant (ein Buch zu lesen).*
- **Subjektsatz (`csubj`):**  *(ein Buch zu lesen) ist gut.*
- **Adverbialsatz (`advcl`):**  *(um ein Buch zu lesen), macht sie früher Schluss.*
- **Attributsatz (adnominaler Satz `acl`):** *sie hat den Plan, (ein Buch zu lesen).*


### *Konstituentenanalyse:*

In [11]:
tree_str = '(ROOT (S (NP (PRP I)) (VP (VBP want) (S (VP (TO to) (VP (VB believe))))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

         ROOT                
          │                   
          S                  
 ┌────────┼────────────────┐  
 │        VP               │ 
 │   ┌────┴────┐           │  
 │   │         S           │ 
 │   │         │           │  
 │   │         VP          │ 
 │   │    ┌────┴─────┐     │  
 NP  │    │          VP    │ 
 │   │    │          │     │  
PRP VBP   TO         VB    . 
 │   │    │          │     │  
 I  want  to      believe  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`VP -> VBP S`

`S -> VP` (Satzregel für subjektlose Infinitivsätze)

`VP -> TO VP`

---
### *Dependenzanalyse:*

In [12]:
sent_nr = """
1 I 2 nsubj  
2 want 0 ROOT
3 to 4 mark
4 believe 2 xcomp
"""

show_dep_trees(sent_nr)

         want(ROOT)               
   ┌─────────┴────────────┐        
   │                believe(xcomp)
   │                      │        
I(nsubj)               to(mark)   



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`want -> believe (xcomp)`

`believe -> to (mark)`

---
### Aufgabe 1c: 

In [13]:
sentence = "the man who knew too much"

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

Relativsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

Attributsatz


### Zusatzfragen:

#### Welche Attributsatztypen haben Sie in der Vorlesung kennengelernt? 

- Relativsatz (`acl:relcl`)
- adnominaler Satz(`acl`): finite oder non-finite

### *Konstituentenanalyse:*

In [14]:
tree_str = '(ROOT (FRAG (NP (NP (DT the) (NN man)) (SBAR (WHNP (WP who)) (S (VP (VBD knew) (ADJP (RB too) (JJ much))))))))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

            ROOT                        
             │                           
            FRAG                        
             │                           
             NP                         
     ┌───────┴─────────┐                 
     │                SBAR              
     │       ┌─────────┴────┐            
     │       │              S           
     │       │              │            
     │       │              VP          
     │       │    ┌─────────┴───┐        
     NP     WHNP  │            ADJP     
 ┌───┴───┐   │    │         ┌───┴────┐   
 DT      NN  WP  VBD        RB       JJ 
 │       │   │    │         │        │   
the     man who  knew      too      much



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`NP -> NP SBAR`

`SBAR -> WHNP S`

---
### *Dependenzanalyse:*

In [15]:
sent_nr = """
1 the 2 det
2 man 0 ROOT
3 who 4 nsubj
4 knew 2 acl:relcl
5 too 6 advmod
6 much 4 advmod
"""

show_dep_trees(sent_nr)

         man(ROOT)                              
   ┌─────────┴─────────────┐                     
   │                knew(acl:relcl)             
   │         ┌─────────────┴─────────────┐       
   │         │                      much(advmod)
   │         │                           │       
the(det) who(nsubj)                 too(advmod) 



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`man -> knew (acl:relcl)`

`knew -> who (nsubj)`

---
### Aufgabe 1d: 

In [16]:
sentence = "They left, but they came back."

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

koordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

keiner (koordinierte Sätze)

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

2 Hauptsätze

### *Konstituentenanalyse:*

In [17]:
tree_str = '(ROOT (S (S (NP (PRP They)) (VP (VBD left))) (, ,) (CC but) (S (NP (PRP they)) (VP (VBD came) (ADVP (RB back)))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                  ROOT                       
                   │                          
                   S                         
      ┌────────┬───┼─────────┬─────────────┐  
      │        │   │         S             │ 
      │        │   │    ┌────┴────┐        │  
      S        │   │    │         VP       │ 
 ┌────┴───┐    │   │    │    ┌────┴───┐    │  
 NP       VP   │   │    NP   │       ADVP  │ 
 │        │    │   │    │    │        │    │  
PRP      VBD   ,   CC  PRP  VBD       RB   . 
 │        │    │   │    │    │        │    │  
They     left  ,  but  they came     back  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`S -> S , CC S .`

---
### *Dependenzanalyse:*

In [18]:
sent_nr = """
1 they 2 nsubj
2 left 0 ROOT
3 but 5 cc
4 they 5 nsubj
5 came 2 conj
6 back 5 advmod
"""

show_dep_trees(sent_nr)

                    left(ROOT)                         
     ┌──────────────────┴───────────┐                   
     │                          came(conj)             
     │         ┌────────────────────┼───────────┐       
they(nsubj) but(cc)            they(nsubj) back(advmod)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`left -> came (conj)`

`came -> but (cc)`



---
### Aufgabe 1e: 

In [19]:
sentence = "This can't be right."

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

keines von beiden

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

keine Einbettung

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

(Satz mit komplexer VP)

### *Konstituentenanalyse:*

In [20]:
tree_str = "(ROOT (S (NP (DT This)) (VP (MD ca) (RB n't) (VP (VB be) (ADJP (JJ right)))) (. .)))"
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

         ROOT                  
          │                     
          S                    
 ┌────────┼──────────────────┐  
 │        VP                 │ 
 │    ┌───┼────────┐         │  
 │    │   │        VP        │ 
 │    │   │    ┌───┴────┐    │  
 NP   │   │    │       ADJP  │ 
 │    │   │    │        │    │  
 DT   MD  RB   VB       JJ   . 
 │    │   │    │        │    │  
This  ca n't   be     right  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`VP -> MD RB VP`

`VP -> VB ADJP` (Kopula-VP)

---
### *Dependenzanalyse:*

Negation:
- `advmod`, vgl. https://universaldependencies.org/de/dep/advmod.html
- oder sprachspezifischer zusätzlicher `neg`-Marker: https://universaldependencies.org/docs/u/dep/neg.html

In [21]:
sent_nr = """
1 this 5 nsubj
2 ca 5 aux
3 n\'t 5 neg
4 be 5 cop
5 right 0 ROOT
"""

show_dep_trees(sent_nr)

            right(ROOT)                 
     ┌───────────┼─────────┬────────┐    
this(nsubj)   ca(aux)   n't(neg) be(cop)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`right -> ca (aux)`

`right -> be (cop)`


---
### Aufgabe 1f: 

In [22]:
sentence = "When the storm started, she was still outside."

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

subordiniert

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

Temporalsatz 

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

Adverbialsatz

### *Konstituentenanalyse:*

In [23]:
tree_str = '(ROOT (S (SBAR (WHADVP (WRB When)) (S (NP (DT the) (NN storm)) (VP (VBD started)))) (, ,) (NP (PRP she)) (VP (VBD was) (ADVP (RB still)) (ADVP (RB outside))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                              ROOT                          
                               │                             
                               S                            
            ┌──────────────────┼────┬────────┬────────────┐  
           SBAR                │    │        │            │ 
  ┌─────────┴─────┐            │    │        │            │  
  │               S            │    │        VP           │ 
  │         ┌─────┴──────┐     │    │   ┌────┼──────┐     │  
WHADVP      NP           VP    │    NP  │   ADVP   ADVP   │ 
  │     ┌───┴─────┐      │     │    │   │    │      │     │  
 WRB    DT        NN    VBD    ,   PRP VBD   RB     RB    . 
  │     │         │      │     │    │   │    │      │     │  
 When  the      storm started  ,   she was still outside  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`S -> SBAR , NP VP .`

`SBAR -> WHADVP S`

---
### *Dependenzanalyse:*

In [24]:
sent_nr = """
1 when 4 mark
2 the 3 det
3 storm 4 nsubj
4 started 6 advcl
5 she 6 nsubj
6 was 0 ROOT
7 still 6 advmod
8 outside 6 advmod 
"""

show_dep_trees(sent_nr)

                            was(ROOT)                                          
    ┌────────────┬──────────────┼─────────────────────────┐                     
    │            │              │                   started(advcl)             
    │            │              │            ┌────────────┴─────────────┐       
    │            │              │            │                     storm(nsubj)
    │            │              │            │                          │       
she(nsubj) still(advmod) outside(advmod) when(mark)                  the(det)  



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`was -> started (advcl)`

`started -> when (mark)`



---
### Aufgabe 1g: 

In [25]:
sentence = "To go now would be a mistake."

### Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)

subordiniert 

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)

Infinitiv-Subjektsatz

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)

Komplementsatz

### *Konstituentenanalyse:*

In [26]:
tree_str = '(ROOT (S (S (VP (TO To) (VP (VB go) (ADVP (RB now))))) (VP (MD would) (VP (VB be) (NP (DT a) (NN mistake)))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                  ROOT                            
                   │                               
                   S                              
     ┌─────────────┴────────┬───────────────────┐  
     S                      │                   │ 
     │                      │                   │  
     VP                     VP                  │ 
 ┌───┴───┐         ┌────────┴───┐               │  
 │       VP        │            VP              │ 
 │   ┌───┴───┐     │    ┌───────┴───┐           │  
 │   │      ADVP   │    │           NP          │ 
 │   │       │     │    │       ┌───┴─────┐     │  
 TO  VB      RB    MD   VB      DT        NN    . 
 │   │       │     │    │       │         │     │  
 To  go     now  would  be      a      mistake  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

`S -> S VP .`

`S -> VP`

`VP -> TO VP`


---
### *Dependenzanalyse:*

In [27]:
sent_nr = """
1 to 2 mark
2 go 7 csubj
3 now 2 advmod
4 would 7 aux
5 be 7 cop
6 a 7 det
7 mistake 0 ROOT
"""

show_dep_trees(sent_nr)

                   mistake(ROOT)                               
    ┌─────────┬──────────┼────────────────────┐                 
    │         │          │                go(csubj)            
    │         │          │          ┌─────────┴──────────┐      
would(aux) be(cop)     a(det)    to(mark)           now(advmod)



#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

`mistake -> go (csubj)`

`go -> to (mark)`

---
## Aufgabe 2: Stellungsfeldermodell

#### Zerlegen Sie den folgenden Satz in Satzklammern und Stellungsfelder. 

#### Analysieren Sie sowohl den Haupt- als auch den Nebensatz auf diese Weise.

*Der kleine Junge hat heute das Buch gelesen, das ihm sein Vater zu Weihnachten geschenkt hat.*

 | | Vorfeld          | linke SK | Mittelfeld                    | rechte SK | Nachfeld              |
|------------|------------------|----------|-------------------------------|-----------|-----------------------|
|***HAUPTSATZ (V2):*** | Der kleine Junge | hat      | heute das Buch                | gelesen,  | NEBENSATZ |
|***NEBENSATZ (VE):***| -                | das      | ihm sein Vater zu Weihnachten | geschenkt hat. | -             

##### Der Nebensatz steht im Nachfeld des Hauptsatzes!